In [1]:
from flask import Flask, jsonify, abort, make_response
from flask_restful import Api, Resource, reqparse, fields, marshal
from flask_httpauth import HTTPBasicAuth
import MySQLdb
import hashlib
from datetime import datetime, timedelta

In [2]:
app = Flask(__name__)
api = Api(app)
auth = HTTPBasicAuth()

In [3]:
db=MySQLdb.connect('localhost', 'root', 'root')
cursor = db.cursor()

In [4]:
# cursor.execute("DROP DATABASE IF EXISTS `yourlife`;")

In [5]:
def makeUserInfoTable():
    # UserInfo Table
    sql = '''CREATE TABLE IF NOT EXISTS `yourlife`.`UserInfo` ( 
        `userid` INT NOT NULL AUTO_INCREMENT , 
        `username` VARCHAR(64) NOT NULL , 
        `password` VARCHAR(64) NOT NULL , 
        `email` VARCHAR(64) NOT NULL , 
        `registered_date` DATE NOT NULL , 
        `activation_key` VARCHAR(64) , 
        PRIMARY KEY (`userid`),
        UNIQUE (`username`),
        UNIQUE (`email`)
        ) ENGINE = InnoDB;
       '''
    cursor.execute(sql)

In [6]:
def makeInGameProfileTable():
    # InGameProfile
    sql = '''CREATE TABLE IF NOT EXISTS `yourlife`.`InGameProfile` ( 
        `userid` INT NOT NULL AUTO_INCREMENT , 
        `job` TINYINT NOT NULL DEFAULT 1, 
        `health` TINYINT NOT NULL DEFAULT 100, 
        `mana` TINYINT NOT NULL DEFAULT 100, 
        `exp` INT NOT NULL DEFAULT 0, 
        `level` TINYINT NOT NULL DEFAULT 1, 
        `stats` TINYINT NOT NULL DEFAULT 0, 
        `party` INT NOT NULL DEFAULT 0, 
        `guild` INT NOT NULL DEFAULT 0, 
        `community` INT NOT NULL DEFAULT 0, 
        `current_quest` INT NOT NULL DEFAULT 0, 
        `location` INT NOT NULL DEFAULT 0, 
        PRIMARY KEY (`userid`)) ENGINE = InnoDB;
       '''
    cursor.execute(sql)

In [7]:
def makeToDoTable():
    # ToDo tambahkan kapan todo dibuat
    sql = '''CREATE TABLE IF NOT EXISTS `yourlife`.`ToDo` ( 
        `userid` INT NOT NULL , 
        `todoid` INT NOT NULL AUTO_INCREMENT , 
        `title` TEXT NOT NULL , 
        `status` TINYINT NOT NULL , 
        `desc` TEXT , 
        `datestart` DATE, 
        `dateline` DATE , 
        `type` INT NOT NULL , 
        `tag` TEXT NOT NULL , 
        `parent` INT, 
        `haschild` BIT,
        PRIMARY KEY (`todoid`), 
        INDEX `idx_userid` (`userid`)) ENGINE = InnoDB;

        '''
    cursor.execute(sql)

In [8]:
print("Preparing Database")

cursor.execute("CREATE DATABASE IF NOT EXISTS yourlife")
makeUserInfoTable()
makeInGameProfileTable()
makeToDoTable()
cursor.execute('USE `yourlife`')

print("Done")

Preparing Database
Done


/home/tegar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: Warning: (1007, "Can't create database 'yourlife'; database exists")
  This is separate from the ipykernel package so we can avoid doing imports until
/home/tegar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: Warning: (1050, "Table 'UserInfo' already exists")
  from ipykernel import kernelapp as app
/home/tegar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: Warning: (1050, "Table 'InGameProfile' already exists")
/home/tegar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: Warning: (1050, "Table 'ToDo' already exists")


In [9]:
def sha256(password):
    password = password.encode('UTF-8')
    hash_object = hashlib.sha256(password)
    hex_dig = hash_object.hexdigest()
    return hex_dig

In [10]:
def md5(text):
    from random import randint
    salt = str(randint(1, 7829))
    text = salt+text
    text = text.encode('UTF-8')
    hash_object = hashlib.md5(text)
    hex_dig = hash_object.hexdigest()
    return hex_dig

In [11]:
def check_not_registered(username, email):
    print(username,email)
    r = cursor.execute("select username, email, activation_key from UserInfo where `username`=%s or `email`=%s;", (username,email))
    if (r > 0):
        r = cursor.fetchone()
        print(r)
        if (r[0] == username):
            print('Username already used')
        elif (r[1] == email):
            print('Email already used')
        return False
    
    return True

In [12]:
def register(username, password, email):
    if check_not_registered(username, email):
        password = sha256(password)
        activation_key = md5(email + str(datetime.utcnow()))
        cursor.execute("INSERT INTO `UserInfo` VALUES (NULL, %s, %s, %s, CURRENT_DATE(), %s)", (username, password, email, activation_key) )
        print('Account activation code: "%s" for %s' % (activation_key, username))
    
        db.commit()

In [13]:
def activation(activation_key, username):
    cursor.execute("select userid, username from UserInfo where `activation_key`=%s;", (activation_key,))
    query = cursor.fetchone()
    try:
        userid = query[0]
        username = query[1]
        cursor.execute("UPDATE UserInfo SET activation_key = NULL WHERE `activation_key`=%s;", (activation_key,))
        db.commit()

        print('Activation success')
        print('Creating ' + username + ' InGameProfile')
        cursor.execute("INSERT INTO `InGameProfile` (userid) VALUES (%s)", (userid,))

        db.commit()
        print(username + ' created. Enjoy!')
        
    except TypeError:
        print("Error: Activation code not valid")

In [14]:
@auth.get_password
def get_password(username):
    cursor.execute("select password from UserInfo where `username`=%s;", (username,))
    query = cursor.fetchone()
    if query != None:
        return query[0]
    return None

@auth.hash_password
def hash_pw(password):
    return sha256(password)

@auth.error_handler
def unauthorized():
    # return 403 instead of 401 to prevent browsers from displaying the default
    # auth dialog
    return make_response(jsonify({'message': 'Unauthorized access'}), 403)

In [15]:
tasks = [
    {
        'id': 1,
        'title': u'Buy groceries',
        'description': u'Milk, Cheese, Pizza, Fruit, Tylenol',
        'done': False
    },
    {
        'id': 2,
        'title': u'Learn Python',
        'description': u'Need to find a good Python tutorial on the web',
        'done': False
    }
]

task_fields = {
    'title': fields.String,
    'description': fields.String,
    'done': fields.Boolean,
    'uri': fields.Url('task')
}

In [16]:
class Register(Resource):
    def __init__(self):
        self.reqparse = reqparse.RequestParser()
        self.reqparse.add_argument('username', type=str, required=True,
                                   help='Your unique username',
                                   location='json')
        self.reqparse.add_argument('password', type=str, required=True,
                                   help='Your password',
                                   location='json')
        self.reqparse.add_argument('email', type=str, required=True,
                                   help='Your email',
                                   location='json')
        super(Register, self).__init__()
        
    def post(self):
        args = self.reqparse.parse_args()
        username = args['username']
        email = args['email']
        password = args['password']
        if check_not_registered(username, email):
            password = sha256(password)
            activation_key = md5(email + str(datetime.utcnow()))
            cursor.execute("INSERT INTO `UserInfo` VALUES (NULL, %s, %s, %s, CURRENT_DATE(), %s)", (username, password, email, activation_key) )
            print('Account activation code: "%s" for %s' % (activation_key, username))

            db.commit()
            return {'status':'success','activation_key': activation_key, 'username': username }, 201
        
        return {'status':'Username already used'}, 409

In [17]:
class TaskListAPI(Resource):
    decorators = [auth.login_required]

    def __init__(self):
        self.reqparse = reqparse.RequestParser()
        self.reqparse.add_argument('title', type=str, required=True,
                                   help='No task title provided',
                                   location='json')
        self.reqparse.add_argument('description', type=str, default="",
                                   location='json')
        super(TaskListAPI, self).__init__()

    def get(self):
        return {'tasks': [marshal(task, task_fields) for task in tasks]}

    def post(self):
        args = self.reqparse.parse_args()
        task = {
            'id': tasks[-1]['id'] + 1,
            'title': args['title'],
            'description': args['description'],
            'done': False
        }
        tasks.append(task)
        return {'task': marshal(task, task_fields)}, 201

In [18]:
class TaskAPI(Resource):
    decorators = [auth.login_required]

    def __init__(self):
        self.reqparse = reqparse.RequestParser()
        self.reqparse.add_argument('title', type=str, location='json')
        self.reqparse.add_argument('description', type=str, location='json')
        self.reqparse.add_argument('done', type=bool, location='json')
        super(TaskAPI, self).__init__()

    def get(self, id):
        task = [task for task in tasks if task['id'] == id]
        if len(task) == 0:
            abort(404)
        return {'task': marshal(task[0], task_fields)}

    def put(self, id):
        task = [task for task in tasks if task['id'] == id]
        if len(task) == 0:
            abort(404)
        task = task[0]
        args = self.reqparse.parse_args()
        for k, v in args.items():
            if v is not None:
                task[k] = v
        return {'task': marshal(task, task_fields)}

    def delete(self, id):
        task = [task for task in tasks if task['id'] == id]
        if len(task) == 0:
            abort(404)
        tasks.remove(task[0])
        return {'result': True}

In [19]:
api.add_resource(Register, '/todo/api/v1.0/register', endpoint='register')
api.add_resource(TaskListAPI, '/todo/api/v1.0/tasks', endpoint='tasks')
api.add_resource(TaskAPI, '/todo/api/v1.0/tasks/<int:id>', endpoint='task')

In [20]:
if __name__ == '__main__':
    app.run(debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Mar/2018 22:27:38] "POST /todo/api/v1.0/register HTTP/1.1" 201 -


bukantegar tegar@iman.name
Account activation code: "4dbf2d8ee1d78daa7f2ef256af6fc64f" for bukantegar


127.0.0.1 - - [21/Mar/2018 22:27:48] "POST /todo/api/v1.0/register HTTP/1.1" 409 -


bukantegar tegar@iman.name
('bukantegar', 'tegar@iman.name', '4dbf2d8ee1d78daa7f2ef256af6fc64f')
Username already used


127.0.0.1 - - [21/Mar/2018 22:28:52] "GET /todo/api/v1.0/task HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2018 22:29:14] "GET /todo/api/v1.0/tasks HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 22:29:47] "GET /todo/api/v1.0/tasks/3 HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2018 22:29:57] "GET /todo/api/v1.0/task/3 HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2018 22:30:01] "GET /todo/api/v1.0/task/2 HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2018 22:30:06] "GET /todo/api/v1.0/task/1 HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2018 22:30:16] "GET /todo/api/v1.0/tasks HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 22:30:36] "GET /todo/api/v1.0/tasks/1 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 22:30:49] "GET /todo/api/v1.0/tasks/2 HTTP/1.1" 200 -


In [21]:
username = 'hello'
password = 'inipassword'
email = 'egar.imansyh@gmail.com'

In [22]:
register(username, password, email)

hello egar.imansyh@gmail.com
('hello', 'egar.imansyh@gmail.com', None)
Username already used


In [23]:
activation('896544a662869b6cbe9da9ffef28905d', username)

Error: Activation code not valid
